# Modulo 24 Atividade 02
#### João Paulo Costa

## AdaBoost e GBM

#### 1. Cinco diferenças entre o AdaBoost e o GBM

| Aspecto                             | **AdaBoost**                                                         | **GBM (Gradient Boosting Machine)**                                                      |
| ----------------------------------- | -------------------------------------------------------------------- | ---------------------------------------------------------------------------------------- |
| **Função de perda (loss function)** | Usa basicamente o *exponencial loss* (mais voltado a classificação). | Pode usar várias funções de perda (ex: deviance, MSE, MAE), o que o torna mais flexível. |
| **Atualização dos pesos**           | Ajusta pesos das amostras mal classificadas a cada iteração.         | Ajusta o modelo pela direção do gradiente negativo do erro da função de perda.           |
| **Modelo base (weak learner)**      | Tipicamente usa *árvores rasas (stumps)*.                            | Usa árvores mais profundas, podendo ajustar `max_depth` para maior complexidade.         |
| **Sensibilidade a ruído**           | Muito sensível a outliers, pois aumenta pesos de erros grandes.      | Mais robusto, pois a otimização via gradiente tende a suavizar erros extremos.           |
| **Otimização**                      | Baseada em reponderação das amostras.                                | Baseada em *gradiente descendente* no espaço dos modelos.                                |

#### 2. Exemplo de classificação e regressão com GBM

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1. Carregar dados
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 2. Criar e treinar modelo
gbm = GradientBoostingClassifier(
    n_estimators=100, 
    learning_rate=0.1, 
    max_depth=3, 
    random_state=42
)
gbm.fit(X_train, y_train)

# 3. Prever e avaliar
y_pred = gbm.predict(X_test)
print("Acurácia:", accuracy_score(y_test, y_pred))
print("\nRelatório de classificação:\n", classification_report(y_test, y_pred))

Acurácia: 1.0

Relatório de classificação:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



#### 3. Cinco hiperparâmetros importantes do GBM

| Hiperparâmetro  | Descrição                                                                          |
| --------------- | ---------------------------------------------------------------------------------- |
| `n_estimators`  | Número de árvores a serem criadas (iterações de boosting).                         |
| `learning_rate` | Taxa de aprendizado (shrinkage) que controla o peso de cada árvore.                |
| `max_depth`     | Profundidade máxima das árvores (controla complexidade e overfitting).             |
| `subsample`     | Proporção dos dados usados em cada iteração (para Stochastic GBM).                 |
| `loss`          | Função de perda a ser minimizada (ex: “deviance”, “exponential”, “squared_error”). |

#### 4. GridSearchCV para otimização dos hiperparâmetros

In [3]:
from sklearn.model_selection import GridSearchCV

# Grade de parâmetros
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [2, 3, 4],
    'subsample': [0.8, 1.0]
}

gbm = GradientBoostingClassifier(random_state=42)

grid = GridSearchCV(
    gbm, 
    param_grid, 
    cv=5, 
    scoring='accuracy', 
    n_jobs=-1
)
grid.fit(X_train, y_train)

print("Melhores parâmetros:", grid.best_params_)
print("Melhor acurácia:", grid.best_score_)

Melhores parâmetros: {'learning_rate': 0.01, 'max_depth': 2, 'n_estimators': 100, 'subsample': 1.0}
Melhor acurácia: 0.9523809523809523


#### 5. Diferença entre GBM e Stochastic GBM

| Característica           | GBM tradicional                                         | Stochastic GBM                                                                |
| ------------------------ | ------------------------------------------------------- | ----------------------------------------------------------------------------- |
| **Amostragem dos dados** | Usa todo o conjunto de dados em cada iteração.          | Usa uma **amostra aleatória (subsample < 1)** em cada iteração.               |
| **Objetivo**             | Minimizar erro global com base em todas as observações. | Introduzir *aleatoriedade* para reduzir overfitting e melhorar generalização. |
| **Vantagem**             | Pode sobreajustar se `n_estimators` for alto.           | Melhora desempenho em dados ruidosos e reduz variância.                       |